# 광고 모델 키워드 추출 코드
- 뉴진스 vs 블랙핑크 케이스를 비교 진행함
- 키워드 형태소는 형용사(VA), 어근(XR) 기준으로 추출 및 분석함

## 1) 웹 크롤링으로 비정형 데이터 수집
- 23년 5~7월 총 3개월 간의 뉴진스와 블랙핑크의 뉴스, 블로그 자료 수집
- 모델별로 하나의 텍스트화

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/멀티캠퍼스 자료/Machine Learning/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

#### 뉴진스 데이터

In [ ]:
train_1 = pd.read_csv(f"{DATA_PATH}뉴진스_뉴스_05.csv")
train_2 = pd.read_csv(f"{DATA_PATH}뉴진스_뉴스_06.csv")
train_3 = pd.read_csv(f"{DATA_PATH}뉴진스_뉴스_07.csv")
train_4 = pd.read_csv(f"{DATA_PATH}뉴진스_블로그_3개월.csv", encoding='cp949')

In [ ]:
train_1.shape , train_2.shape , train_3.shape , train_4.shape

((1030, 4), (912, 4), (595, 4), (148, 4))

In [ ]:
train_nj = pd.concat([train_1,train_2,train_3,train_4],axis=0)
train_nj.head()

,date,title,link,content,Unnamed: 0,datetime
0,2023.05.31. 오전 10:06,\n\t\t\t'그래미 5관왕' 존 바티스트 신곡에 뉴진스 협업\n\t\t,https://n.news.naver.com/mnews/article/003/001...,[\n'코크 스튜디오' 위해 만든 싱글 '비 후 유 아' 작업 \n[서울=뉴시스] ...,NaN,NaN
1,2023.05.31. 오후 3:31,"\n\t\t\t""'품위녀'와 전혀 달라"" K-맘 뉴진스의 '행복배틀'[종합]\n\t\t",https://n.news.naver.com/mnews/article/437/000...,[\n \n31일 오후 서울 마포구 상암동 스탠포드호텔에서 ENA '행복배틀'...,NaN,NaN
2,2023.05.31. 오후 3:40,"\n\t\t\t뉴진스, 블랙핑크 그리고 ‘행복배틀’…이엘·진서연→우정원, 케이맘이 ...",https://n.news.naver.com/mnews/article/057/000...,"[\n \n‘행복배틀’ 제작발표회 사진=김영구 MK스포츠 기자‘행복배틀’ 이엘, 진...",NaN,NaN
3,2023.05.31. 오후 3:16,"\n\t\t\t""뉴진스·블랙핑크 이전에 K-맘"" 진서연→이엘 '행복배틀' 발발[종합...",https://n.news.naver.com/mnews/article/108/000...,[\n\t\t\t\t\n\t\t\t\t\n\t\t\t\t[스타뉴스 | 상암(서울)=...,NaN,NaN
4,2023.05.31. 오후 6:25,\n\t\t\t뉴진스·아이브·에스파 이어 (여자)아이들…'퀸카' 멜론 일간차트 1위...,https://n.news.naver.com/mnews/article/003/001...,"[\n미니 6집 '아이 필', '빌보드 200' 41위 자체 최고 기록 \n[서울=...",NaN,NaN


In [ ]:
train_nj['title'] = train_nj['title'].str.replace("[^a-zA-Z가-힣ㄱ-ㅎ0-9.,!? ]" , "",regex=True)
train_nj['content'] = train_nj['content'].str.replace("[^a-zA-Z가-힣ㄱ-ㅎ0-9.,!? ]" , "",regex=True)

# 제목, 본문내용 합치기
nj_text = train_nj['title'] + train_nj['content']
nj_text

0      그래미 5관왕 존 바티스트 신곡에 뉴진스 협업코크 스튜디오 위해 만든 싱글 비 후 ...
1      품위녀와 전혀 달라 K맘 뉴진스의 행복배틀종합    31일 오후 서울 마포구 상암동...
2      뉴진스, 블랙핑크 그리고 행복배틀이엘진서연우정원, 케이맘이 온다종합M이슈 행복배틀 ...
3      뉴진스블랙핑크 이전에 K맘 진서연이엘 행복배틀 발발종합스타뉴스  상암서울김노을 기자...
4      뉴진스아이브에스파 이어 여자아이들퀸카 멜론 일간차트 1위미니 6집 아이 필, 빌보드...
                             ...                        
143    뉴진스 토끼티  포토카드  앨범기밀이라고 해서, 거참 어디가서 말할 수는 없지만,뉴...
144    오렌즈 프렌츠샤인 원데이 웜톤  쿨톤을 위한 뉴진스 렌즈오렌즈에서 프렌츠샤인이라는신...
145    뉴진스 파워퍼프걸 캐릭터 만들기 테스트 링크 mbti 의외로 ENFJ미국의 키치한 ...
146    뉴진스의 사인 축구 셔츠를 받을 수 있는 이벤트 공개  with 카포스토어대한민국의...
147    뉴진스 파워퍼프걸 콜라보 케이스티파이 powerpuff yourself파워퍼프걸 뉴...
Length: 2685, dtype: object

### 블랙핑크 데이터

In [ ]:
train_1 = pd.read_csv(f"{DATA_PATH}블랙핑크_뉴스_5월.csv")
train_2 = pd.read_csv(f"{DATA_PATH}블랙핑크_뉴스_6월.csv")
train_3 = pd.read_csv(f"{DATA_PATH}블랙핑크_뉴스_7월.csv")
train_4 = pd.read_csv(f"{DATA_PATH}블랙핑크_블로그_3개월.csv", encoding='utf-8')

In [ ]:
train_1.shape , train_2.shape , train_3.shape , train_4.shape

((1125, 4), (387, 4), (235, 4), (180, 4))

In [ ]:
train_bp = pd.concat([train_1,train_2,train_3,train_4],axis=0)
train_bp.head()

,date,title,link,content,Unnamed: 0,datetime
0,2023.05.31. 오후 9:17,\n\t\t\t블랙핑크와 게임의 만남\n\t\t,https://n.news.naver.com/mnews/article/469/000...,"[\n'블랙핑크 더 게임', 지난 18일 글로벌 출시""블랙핑크 매력 보여줄 수 있는...",NaN,NaN
1,2023-05-31 20:00:01,[쉬운 우리말 쓰기] 블랙핑크가 맡은 '헤드라이너'가 뭔가요?,https://n.news.naver.com/mnews/article/001/001...,[\n\t\t\t[쉬운 우리말 쓰기] 블랙핑크가 맡은 '헤드라이너'가 뭔가요?[ ※...,NaN,NaN
2,2023-05-30 08:29:03,"유안타證 ""와이지엔터, 블랙핑크 성장세 지속""",https://n.news.naver.com/mnews/article/003/001...,[\n\n\n\n\n(자료제공=유안타증권) *재판매 및 DB 금지[서울=뉴시스]이지...,NaN,NaN
3,2023.05.30. 오전 9:05,"\n\t\t\t'블랙핑크 여동생' 베이비 몬스터, 심상치 않은 인기..글로벌팬 공감...",https://n.news.naver.com/mnews/article/112/000...,[\n\t\t\t\t\n\t\t\t\t\n\t\t\t\t[헤럴드POP=김지혜 기자]...,NaN,NaN
4,2023.05.31. 오후 3:11,"\n\t\t\t""블랙핑크·뉴진스 전에 K맘""…'행복배틀' 이엘→진서연의 스릴러 [종...",https://n.news.naver.com/mnews/article/311/000...,[\n \n(엑스포츠뉴스 김현정 기자) ‘행복배틀’이 치열한 심리전과 예측불가 전개...,NaN,NaN


In [ ]:
train_bp['title'] = train_bp['title'].str.replace("[^a-zA-Z가-힣ㄱ-ㅎ0-9.,!? ]" , "",regex=True)
train_bp['content'] = train_bp['content'].str.replace("[^a-zA-Z가-힣ㄱ-ㅎ0-9.,!? ]" , "",regex=True)

# 제목, 본문내용 합치기
bp_text = train_bp['title'] + train_bp['content']
bp_text

0      블랙핑크와 게임의 만남블랙핑크 더 게임, 지난 18일 글로벌 출시블랙핑크 매력 보여...
1      쉬운 우리말 쓰기 블랙핑크가 맡은 헤드라이너가 뭔가요?쉬운 우리말 쓰기 블랙핑크가 ...
2      유안타 와이지엔터, 블랙핑크 성장세 지속자료제공유안타증권 재판매 및 DB 금지서울뉴...
3      블랙핑크 여동생 베이비 몬스터, 심상치 않은 인기..글로벌팬 공감대헤럴드POP김지혜...
4      블랙핑크뉴진스 전에 K맘행복배틀 이엘진서연의 스릴러 종합 엑스포츠뉴스 김현정 기자 ...
                             ...                        
175    스타벅스 신메뉴 블랙핑크 콜라보 제품 후기 디저트 먹고 꿀팁스벅앱을 들어가니 뜬 소...
176    블랙핑크 제니 열애설 상기 시그널? 방탄소년단BTS 뷔와 파리 데이트룩 자발적 공개...
177    연예인향수 궁금하지 미스디올 블랙핑크 지수 강민경 엄정화 손석구 향수안녕하세요 다양...
178    스벅신상 블랙핑크 딸기 크림 초코번 맛있네제가 좋아하는 두 조합!!블랙핑크  스타벅...
179    베트남 하노이 블랙핑크 환영2층버스 팬들로 가득베트남 하노이 블랙핑크 환영2층버스 ...
Length: 1927, dtype: object

## 2) 용어 리스트 구축
- 1차 - KKM 세종 말뭉치 기준, 형태소별 통계 전체횟수 5회 이상 단어들 중 '인물'에 대한 형태소로 제한하여 단어 추출
(http://kkma.snu.ac.kr/statistic)
 - 총 1825개 형용사(VA) 중 512개 추출
 - 총 2643개 어근(XR) 중 681개 추출

- 2차 - 웹 크롤링 데이터 중 제외된 형태소 추가 보완하여 최종 리스트 구축함
 - 최종 1476개의 인물 형태소 리스트
 - (형용사: 637개, 어근: 839개)




### 1차 - KKM에서 유의미 단어 추출
- 형용사(VA) : va_meaning_words
- 어근(XR) : xr_meaning_words

In [ ]:
va_meaning_words = ['크', '좋','새롭','다르','어렵','쉽','작','길','아름답','어리','힘들','젊','아프','빠르','나쁘','싫','무섭','짧',
                 '커다랗','엄청나','지나치','희','검','어둡','옳','괜찮','밝','바쁘','가볍','뜨겁','하얗','예쁘','무겁','맑','즐겁','부드럽',
                 '재미있','약하','낯설','편하','낫','비싸','강하','슬프','춥','놀랍','곱','뛰어나','낡','거칠','안타깝','부끄럽','기쁘','바르',
                 '두렵','반갑','흔하','착하','차갑','싸','조그맣','외롭','올바르','빨갛','멋지','날카롭','급하','친하','굵','굳','아쉽',
                 '느리','환하','더럽','그립','괴롭','귀엽','우습','가늘','어리석','귀하','둥글','부럽','노랗','귀찮','재밌','세','까맣','두껍',
                 '어지럽','거세','차','흐리','파랗','밉','아깝','시끄럽','눈부시','얇','진하','힘겹','점잖','고프','까다롭','힘차','폭넓',
                 '못하','잘나','사납','두드러지','끈질기','험하', '이쁘',' 누렇',' 시커멓','오래','낯익','배고프','끊임없','남다르','지겹',
                 '비','쓸데없','못나','벅차','묘하','기막히','서럽','서글프','세차','값싸','그르','모질','가엾','잘','참되','허옇','엷',
                 '색다르','동그랗','게으르','메마르','벌겋','쑥스럽','엄하','어이없','가느다랗','너그럽','서투르','하찮','시리','매끄럽',
                 '우스꽝스럽','고달프','서툴','뻔하','값지','멋있','잘생기','질기','멍하','새까맣','연하','추하','틀리','억세','안되','여리',
                 '싱겁','쓰라리','어처구니없','역겹','재미없','수줍','뿌옇','천하','징그럽','순하','흉하','안쓰럽','야하','빼어나','속상하',
                 '유별나','매섭','보잘것없','못생기','못되','시퍼렇','기다랗','짓궂','언짢','잽싸','형편없','값비싸','흥겹','새파랗','둥그렇',
                 '둔하','부시','재빠르','딱하','배부르','싱그럽','별나','쓰리','야무지','새하얗','어질','한결같','활기차','궂','아리','갸날프',
                 '건방지','독하','선하','훤하','시뻘겋','탁하','무디','우렁차','능하','철없','뽀얗','번거롭','해롭','쓸모없','성가시','정겹',
                 '애처롭','퍼렇','분하','관계없','네모나','얄밉','차디차','장하','이롭','달갑','헤프','앳되','후하','푸르르','쎄','힘세',
                 '힘없','보드랍','새빨갛','길다랗','아니꼽','미심쩍','해맑','미덥','수상쩍','굳세','만만찮','동떨어지','껄끄럽','험상궂',
                 '변함없','드세','앙증맞','깡마르','구슬프','날쌔','실없','쓰디쓰','시꺼멓','을씨년스럽','검푸르','발갛','시원찮','뻘겋',
                 '어여쁘','당차','고되','모나','다부지','흐드러지','드럽','짙푸르','약삭빠르','방정맞','뜸하','사이좋','천연덕스럽','무르',
                 '볼품없','상스럽','새카맣','격하','거침없','허하','희뿌옇','앙칼지','심술궂','아리땁','끈덕지','역하','가녀리','노오랗',
                 '굼뜨','희디희','가엽','살갑','하잘것없','정신없','버릇없','올곧','악하','청승맞','별스럽','구리','샛노랗','길하','빈틈없',
                 '필요없','남부럽','멋들어지','쪼그맣','똥그랗','매몰차','파아랗','속좁','막되','노하','얌전하','얇디얇','얍삽하','연푸르',
                 '번거럽','똘똘하','야물딱지','예쁘장하','투박스럽','게을러빠지','껄렁껄렁하','믿음직스럽','사근사근하','기','될성부르',
                 '약아빠지','예스럽','쇼킹하','놀라웁','맑디맑','능청맞','곱디곱','징하','모지','꽁하','따스','늙','유머러스하','예쁘장스럽',
                 '촌시럽','보오얗','동그렇','모자르','모잘르','간지','힘쎄','자애롭','시덥잖','쌍스럽','쑥쓰럽','앙징맞','밉살맞','후덥',
                 '여물','찐하','찡하','둥굴','기차','자애롭','시덥잖','쌍스럽','쑥쓰럽','앙징맞','얄따랗','밉살맞','버얼겋','높푸르','찐하',
                 '찡하','기차','주제넘','꺼멓','어리숙하','얄궂','참하','끄떡없','희망차','기운차','희부옇','안스럽','어줍잖','의롭','능글맞',
                 '간드러지','쌀쌀맞','자그맣','볼썽사납','희멀겋','도드라지','재수없','걸','빠알갛','뜬금없','살지','악착같','멀겋','의좋','꾸밈없',
                 '같잖','잡스럽','냉하','싸하','빡세','궁상맞','밉살스럽','하이얗','익살맞','쬐그맣','암팡지','어쭙잖','분별없','달갑잖','똑바르',
                 '실하','박하','고깝','특별나','새삼스럽','스스럼없','좀스럽','손색없','후지','짠하','시답잖','허물없','변변찮','따습','맹하',
                 '애닯','주책없','뚱하','의심쩍','쬐끄맣','섹시하','곱다랗','시건방지','특출나','옹골차','막돼먹','곰살궂','가소롭','유하','속없',
                 '지각없','거멓','대단찮','기똥차','깊푸르','따시','뜨시','맥없','싯누렇','로맨틱하','도탑','허여멀겋','모지락스럽','향그럽',
                 '옹골지','성마르','성스럽','꼴사납','시답','동글','마르','대차','조그만하','인상깊','야멸차','염치없','쌩뚱맞','매스껍','극성맞',
                 '깊디깊','감명깊','반하','궁하','덜떨어지','나즈막하','요상하','작디작','째쨰하','촌스럽','뽀오얗','색스럽','다사롭','둔덕지',
                 '맵싸하','까아맣','감미롭','짖궂','시덥','어줍','몽글','값있','눈꼴사납']

In [ ]:
len(va_meaning_words)

512

In [ ]:
xr_meaning_words = ['강력', '훌륭', '바람직', '따뜻', '깨끗', '화려', '궁금', '진정', '진지', '대단', '익숙', '다양', '조용', '시원',
                     '답답', '소중', '활발', '엉뚱', '평범', '엄격', '솔직', '불쌍', '어색', '억울', '절실', '쓸쓸', '생생', '민감',
                     '신선', '강렬', '만만', '끔찍', '지루', '소박', '이상', '단단', '격렬', '한심', '초라', '과감', '튼튼', '유명',
                     '유연', '섬세', '똑똑', '단호', '지독', '섭섭', '차분', '간절', '순진', '애매', '심심', '불쾌', '절박', '은밀',
                     '예민', '깔끔', '싱싱', '달콤', '잔잔', '유쾌', '서늘', '따스', '싸늘', '집요', '성급', '황당', '떳떳', '유치',
                     '지저분', '우아', '과격', '거룩', '위태', '거대', '단정', '느긋', '용감', '저렴', '허전', '든든', '창백', '멍청',
                     '독특', '조급', '급급', '고귀', '생소', '희한', '친근', '어수선', '철저', '거북', '썰렁', '담담', '세심', '경쾌',
                     '허술', '원만', '예리', '야릇', '비겁', '근사', '상쾌', '허름', '진부', '씩씩', '반듯', '비장', '포근', '고약',
                     '고상', '미숙', '정중', '웅장', '아찔', '난해', '안이', '냉혹', '못마땅', '열렬', '고단', '수상', '탄탄', '강인',
                     '뻔뻔', '연약', '아늑', '자그마', '추악', '앙상', '관대', '묵직', '치열', '엉성', '큼직', '순박', '구수', '화사',
                     '험악', '일관', '광활', '애틋', '끈끈', '자상', '무시무시', '삭막', '경건', '발랄', '의젓', '착실', '꾸준', '쌀쌀',
                     '왜소', '처참', '숭고', '날씬', '상냥', '교활', '처량', '산뜻', '영리', '납작', '산만', '무뚝뚝', '애절', '진솔',
                     '얄팍', '뻣뻣', '담백', '아련', '오묘', '훈훈', '말끔', '온화', '침울', '어정쩡', '신랄', '공손', '냉철', '날렵',
                     '아담', '해괴', '투박', '민첩', '육중', '우람', '외람', '묵묵', '근소', '영특', '아둔', '괴팍', '거무스름', '뾰로통',
                     '예쁘장', '졸렬', '온건', '빵빵', '담대', '따듯', '다정다감', '께름칙', '핼쑥', '흉포', '따끈', '달착지근', '서글서글',
                     '어리벙벙', '현란', '홀쭉', '멀뚱', '공명정대', '유약', '끈적', '위태위태', '동글납작', '팔팔', '소소', '살벌', '순진무구',
                     '믿음직', '허약', '느릿', '주도면밀', '둥그스름', '거무스레', '소탈', '미적지근', '호쾌', '걸걸', '누르스름', '우스꽝',
                     '호화', '동그스름', '치명', '심약', '파릇', '저속', '영민', '발칙', '명랑', '미련', '똘똘', '새콤', '완고', '굵직',
                     '소심', '갑갑', '당돌', '당당', '늘씬', '우수', '용', '미진', '빽빽', '의연', '매끈', '음울', '합리', '척박', '근엄',
                     '면밀', '상큼', '노련', '건전', '강경', '푸근', '풍족', '가련', '괘씸', '조잡', '시들', '세련', '헐렁', '누추', '자잘',
                     '활달', '적대', '완연', '늠름', '스산', '숙연', '얼떨떨', '어엿', '자신만만', '어두컴컴', '갸름', '쾌활', '고지식', '방만',
                     '경박', '건장', '꼿꼿', '따분', '초췌', '부당', '왕성', '기특', '미지근', '건실', '밋밋', '어렴풋', '길쭉', '도도',
                     '향긋', '구태의연', '초연', '정갈', '호젓', '수려', '솔깃', '준엄', '시큰둥', '침침', '꿋꿋', '정결', '번듯', '단아',
                     '캄캄', '불온', '시무룩', '빳빳', '비열', '지긋', '꺼림칙', '음탕', '영악', '야비', '냉랭', '노골', '따사', '비범', '찜찜',
                     '대범', '어눌', '흥미진진', '어둑', '영롱', '조촐', '절절', '깜찍', '푸르스름', '세세', '절친', '풍만', '낭자', '유용',
                     '칙칙', '쟁쟁', '기민', '대담', '수수', '심드렁', '진귀', '얼얼', '말쑥', '음침', '삐딱', '옹색', '추잡', '뻐근', '장대',
                     '풋풋', '훤칠', '결연', '냉엄', '뻔', '비릿', '인자', '유력', '방자', '가혹', '다정', '괴이', '야트막', '화창', '애잔',
                     '고고', '굳건', '갸륵', '매정', '아리송', '취약', '덤덤', '온순', '심란', '조악', '장중', '농후', '넓적', '울적', '해박',
                     '둔탁', '무고', '까무잡잡', '무지막지', '촘촘', '원숙', '느끼', '능통', '호탕', '휘황', '음험', '황당무계', '흉흉', '퀴퀴',
                     '강직', '으슥', '격', '저돌', '잔인', '음습', '뚱뚱', '말짱', '빡빡', '괴상', '푹신', '원대', '고즈넉', '꾀죄죄', '반반',
                     '깐깐', '눅눅', '독실', '기구', '망측', '처연', '이슥', '선량', '수척', '황홀', '낭랑', '싹싹', '맹랑', '명석', '꼼꼼',
                     '야박', '듬직', '고매', '걸쭉', '가뿐', '유익', '후줄근', '어수룩', '황폐', '유순', '특출', '통렬', '고리타분', '우직',
                     '섹시', '견실', '강대', '무정', '중후', '깜깜', '비천', '두둑', '둔중', '순전', '희끄무레', '구부정', '무덤덤', '유복',
                     '은근', '불미', '망연', '도톰', '까칠', '요란', '평이', '청명', '퀭', '과', '옹졸', '너저분', '파리', '무던', '묘연',
                     '완곡', '청아', '나약', '능란', '초조', '무모', '엉큼', '어둠침침', '혹심', '처절', '깍듯', '알뜰', '알량', '거무튀튀',
                     '고루', '다소곳', '가무잡잡', '청량', '쾌청', '정밀', '질겁', '아뜩', '아릿', '간명', '으리으리', '정정당당', '청초', '강건',
                     '뜨악', '망망', '등등', '괴괴', '난잡', '간사', '파르스름', '휘황찬란', '용의주도', '부지런', '불그레', '뜨뜻', '진득',
                     '진중', '범상', '펑퍼짐', '고만고만', '편편', '출중', '감미', '소담', '병약', '성성', '각별', '공고', '과대', '가증',
                     '일천', '왁자', '유려', '먹먹', '미천', '미려', '과도', '괄괄', '기기묘묘', '터프', '앙증', '멀쑥', '딴딴', '궁벽',
                     '고결', '고아', '시시껄렁', '어두침침', '이상야릇', '형형', '혼탁', '쩨쩨', '찝찝', '저릿', '신성', '거무죽죽', '휑뎅그렁',
                     '노르스름', '능수능란', '불그죽죽', '편평', '해쓱', '침통', '털털', '토실', '지순', '지엄', '우울', '울울', '신속', '심난',
                     '심원', '살뜰', '뜨끔', '마뜩', '망극', '가열', '찌뿌드드', '호화찬란', '득의만만', '로맨틱', '어리숙', '에로틱', '현격',
                     '쫄깃', '장쾌', '적의', '소홀', '앙큼', '애처', '박정', '범연', '부박', '명철', '무결', '다기', '교교', '극미', '추',
                     '푸르뎅뎅', '푸르죽죽', '감', '드라마틱', '뻑적지근', '사근사근', '애매모호', '어머어마', '무지근', '오종종', '이쁘장', '황막',
                     '흉악', '컬컬', '소략', '소복', '심장', '양순', '무감', '무망', '껌껌', '발그스름', '간구', '덥수룩', '도도록', '부정직',
                     '편벽', '희끗', '척척', '친숙', '자별', '정대', '정숙', '엔간', '오롯', '완서', '위중', '서름', '선뜻', '막강', '망칙',
                     '냉정', '노숙', '우두망찰', '흐리멍덩', '고요', '거만', '암울', '괴상망측', '두리뭉실', '둥글넓적', '불그스름', '불효막심',
                     '살기등등', '시금털털', '그들먹', '판판', '훤출', '천진', '측은', '정치', '역연', '요상', '유일', '숭엄', '식상', '방정',
                     '분망', '말똥', '무계', '괄목', '간악', '걸찍', '고만', '뜨뜻미지근', '쇠']


In [ ]:
len(xr_meaning_words)

681

### 2차 - 단어 보완 및 최종 리스트

- KKM에 없으면서, 데이터에 있었던 단어들 중에서 추가 단어 선별
- 구체적인 2차 리스트 보완 코드는 아래 '참고'란에서 확인 가능



- 보완 단어 리스트

In [ ]:
text_unique_va_keywords = [
    '간절하', '강렬하', '경쾌하', '고고하', '과감하', '과하', '관대하', '굉장하', '교묘하', '구수하', '굳건하', '굵직하', '깐깐하', '깜찍하',
    '깨끗하', '꾸준하', '끈끈하', '나른하', '난감하', '난처하', '난해하', '날렵하', '날씬하', '납작하', '누렇', '느긋하', '느끼하', '능수능란하',
    '능숙하', '단단하', '단정하', '단호하', '달달하', '달콤하', '담담하', '담백하', '답답하', '대견하', '대단하', '든든하', '따듯하', '따뜻하',
    '따스하', '딱딱하', '또렷하', '뚜렷하', '뜨끈하', '뜻깊', '막중하', '매끈하', '매콤하', '무난하', '미미하', '미약하', '밋밋하', '발랄하',
    '발칙하', '복잡하', '불쌍하', '뻔뻔하', '뿌듯하', '상큼하', '새콤하', '생뚱맞', '생생하', '생소하', '서운하', '섬세하', '섭섭하', '세심하',
    '소소하', '수수하', '시원하', '신기하', '신선하', '싸늘하', '씁쓸하', '씩씩하', '애절하', '어눌하', '어색하', '어설프', '얼떨떨하',
    '열악하', '예민하', '오지', '우아하', '웅장하', '위대하', '유연하', '유창하', '잔잔하', '쟁쟁하', '저렴하', '절실하', '절친하', '정갈하',
    '조그마하', '조급하', '지루하', '지저분하', '진솔하', '진지하', '차분하', '참신하', '촉촉하', '친근하', '쾌활하', '쿨하', '큼직하', '탁월하',
    '탄탄하', '투박하', '튼튼하', '평범하', '풋풋하', '핫하', '향긋하', '화기애애하', '화려하', '화사하', '화창하', '활발하', '훈훈하', '훌륭하'
]

In [ ]:
len(text_unique_va_keywords)

125

In [ ]:
text_unique_xr_keywords = [
    '가지런', '거창', '견고', '겸허', '굉장', '교묘', '귀중', '그윽', '기괴', '기발', '기이', '나른', '난감', '넉넉', '널널', '능숙',
    '다채', '단순', '단조', '단출', '달짝지근', '대견', '돈독', '두근', '딱딱', '떠들석', '떠들썩', '떨떠름', '또렷', '뚜렷', '막막',
    '막연', '말랑', '매콤', '맹렬', '머쓱', '멀쩡', '명쾌', '명확', '모호', '몽롱', '무난', '무도', '무색', '뭉근', '뭉클', '미묘', '미미',
    '미세', '미약', '복잡', '복잡다단', '분주', '불순', '비아냥', '사소', '상세', '생경', '서운', '선명', '성대', '세밀', '수더분', '수월',
    '순탄', '술렁', '시끌', '신기', '심각', '심상', '심오', '쌉쌀', '쏠쏠', '씁쓸', '아스라', '어마어마', '어중간', '어지간', '엄', '엄정',
    '열악', '온전', '완만', '용이', '위대', '유창', '으르렁', '은은', '의미심장', '의아', '이룩', '자세', '잠잠', '잡다', '적당', '적절',
    '절묘', '정교', '정당', '정연', '정정', '정확', '조그마', '주요', '준수', '중요', '즐비', '지대', '진기', '짜릿', '짤막', '찬란',
    '참신', '창창', '청청', '촉촉', '축축', '충분', '치밀', '친밀', '칼칼', '쾌적', '쿵쾅', '큼지막', '탁월', '텁텁', '테이크', '톡톡',
    '통통', '특별', '특유', '특이', '팔랑', '팽팽', '편', '평평', '폭신', '풍부', '풍성', '한가', '합당', '험난', '혹독', '홀가분', '화',
    '화기애애', '확고', '확실', '확연', '환', '획일', '후덥지근', '흐릿', '흐뭇', '흔', '희귀', '희미',
]

In [ ]:
len(text_unique_xr_keywords)

158

- 최종 용어리스트 업데이트

In [ ]:
va_meaning_words = va_meaning_words + text_unique_va_keywords
xr_meaning_words = xr_meaning_words + text_unique_xr_keywords

In [ ]:
len(va_meaning_words), len(xr_meaning_words), len(va_meaning_words+xr_meaning_words)

(637, 839, 1476)

In [ ]:
df = va_to_noun(va_meaning_words) + xr_to_noun(xr_meaning_words)
len(set(df))

## 136 단어 정도 어근, 형용사 겹침

1340

## 3) Kiwi로 데이터 토큰화 및 형태소 추출
- 형태소 추출은 형용사(VA), 어근(XR)에 대해서 진행함

In [ ]:
!pip install kiwipiepy

In [ ]:
import kiwipiepy
from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords
stopwords =  Stopwords()
kiwi = Kiwi()

### 뉴진스 - 형태소 (VA, XR) 추출
- 위에 구축한 유의미 단어 리스트에 포함된 형태소만을 추출함

- 형용사(VA)

In [ ]:
nj_text = [str(item) for item in nj_text]

In [ ]:
nj_va_meaningwords =[]

for idx, text in enumerate(nj_text) :
    result = kiwi.tokenize(text,stopwords=stopwords)
    tmp = [token.form for token in result if token.tag in ['VA'] and token.form in va_meaning_words] #형용사
    for idx, word in enumerate(tmp) :
        nj_va_meaningwords.append(word)

In [ ]:
len(nj_va_meaningwords)

7960

- 어근(XR)

In [ ]:
nj_xr_meaningwords =[]

for idx, text in enumerate(nj_text) :
    result = kiwi.tokenize(text,stopwords=stopwords)
    tmp = [token.form for token in result if token.tag in ['XR'] and token.form in xr_meaning_words] #형용사
    for idx, word in enumerate(tmp) :
        nj_xr_meaningwords.append(word)

In [ ]:
len(nj_xr_meaningwords)

4264

### 블랙핑크 - 형태소 (VA, XR) 추출
- 위에 구축한 유의미 단어 리스트에 포함된 형태소만을 추출함

In [ ]:
bp_text = [str(item) for item in bp_text]

- 형용사(VA)

In [ ]:
bp_va_meaningwords =[]

for idx, text in enumerate(bp_text) :
    result = kiwi.tokenize(text,stopwords=stopwords)
    tmp = [token.form for token in result if token.tag in ['VA'] and token.form in va_meaning_words] #형용사
    for idx, word in enumerate(tmp) :
        bp_va_meaningwords.append(word)

In [ ]:
len(bp_va_meaningwords)

3087

- 어근(XR)

In [ ]:
bp_xr_meaningwords =[]

for idx, text in enumerate(bp_text) :
    result = kiwi.tokenize(text,stopwords=stopwords)
    tmp = [token.form for token in result if token.tag in ['XR'] and token.form in xr_meaningwords] #형용사
    for idx, word in enumerate(tmp) :
        bp_xr_meaningwords.append(word)

In [ ]:
len(bp_xr_meaningwords)

1133

## 4) 키워드 명사화
- 형용사(VA), 어근(XR) 형태의 형태소들을 명사형으로 키워드에 적합하게 변형함

### 형용사(VA) to 명사

In [ ]:
def va_to_noun(adjective_list):
    results = []
    for adjective in adjective_list:
        last_char = adjective[-1]

        if (ord(last_char) - 44032) % 28 != 0: #받침이 있다면 음을 더하고
            result = adjective + "음"
        else:
            last_jamo = chr(ord(last_char) + 16) #없다면 마지막 글자에 ㅁ 받침을 추가
            result = adjective[:-1] + last_jamo

        results.append(result)
    return results

In [ ]:
nj_nva_meaningwords = va_to_noun(nj_va_meaningwords)
bp_nva_meaningwords = va_to_noun(bp_va_meaningwords)

nj_nva_meaningwords

['폭넓음',
 '다름',
 '느림',
 '예쁨',
 '다름',
 '재미있음',
 '셈',
 '좋음',
 '좋음',
 '큼',
 '짧음',
 '예쁨',
 '다름',
 '예쁨',
 '재밌음',
 '뻔뻔함',
 '좋음',
 '큼',
 '기쁨',
 '큼',
 '재밌음',
 '강함',
 '다름',
 '좋음',
 '예쁨',
 '재밌음',
 '큼',
 '큼',
 '젊음',
 '재밌음',
 '재미있음',
 '느림',
 '큼',
 '큼',
 '큼',
 '좋음',
 '뻔뻔함',
 '재미있음',
 '다름',
 '좋음',
 '큼',
 '큼',
 '어림',
 '예쁨',
 '큼',
 '큼',
 '짧음',
 '짧음',
 '기쁨',
 '다름',
 '재미있음',
 '좋음',
 '큼',
 '재미있음',
 '힘들음',
 '좋음',
 '재밌음',
 '뻔함',
 '느림',
 '큼',
 '재밌음',
 '철없음',
 '큼',
 '재밌음',
 '좋음',
 '재밌음',
 '다름',
 '예쁨',
 '재밌음',
 '멋짐',
 '예쁨',
 '큼',
 '강렬함',
 '큼',
 '다름',
 '재밌음',
 '어림',
 '좋음',
 '길음',
 '짧음',
 '기쁨',
 '길음',
 '큼',
 '기쁨',
 '강렬함',
 '예쁨',
 '재미있음',
 '재미있음',
 '꾸밈없음',
 '좋음',
 '좋음',
 '좋음',
 '길음',
 '거칠음',
 '힘들음',
 '빠름',
 '큼',
 '끈끈함',
 '길음',
 '서툴음',
 '여림',
 '어림',
 '능글맞음',
 '좋음',
 '바쁨',
 '빠름',
 '빠름',
 '남다름',
 '힘들음',
 '좋음',
 '좋음',
 '격함',
 '큼',
 '묘함',
 '과감함',
 '큼',
 '색다름',
 '큼',
 '색다름',
 '좋음',
 '좋음',
 '밝음',
 '꾸준함',
 '깜찍함',
 '낯설음',
 '큼',
 '어림',
 '좋음',
 '꾸준함',
 '좋음',
 '빠름',
 '기쁨',
 '색다름',
 '빠름',
 '큼',
 '큼',
 '큼',
 '좋음',
 '좋음',
 '큼',

### 어근(XR) to 명사

In [ ]:
def xr_to_noun(xr_list):
    results = []
    for xr in xr_list:
        result = xr + '함'

        results.append(result)
    return results

In [ ]:
nj_nxr_meaningwords = xr_to_noun(nj_xr_meaningwords)
bp_nxr_meaningwords = xr_to_noun(bp_xr_meaningwords)

nj_nxr_meaningwords

['바람직함',
 '차분함',
 '예민함',
 '까칠함',
 '예민함',
 '까칠함',
 '평범함',
 '민감함',
 '차분함',
 '예민함',
 '까칠함',
 '궁금함',
 '신기함',
 '강렬함',
 '든든함',
 '소소함',
 '지독함',
 '머쓱함',
 '신선함',
 '유명함',
 '바람직함',
 '다채함',
 '예민함',
 '궁금함',
 '지루함',
 '확고함',
 '따뜻함',
 '고즈넉함',
 '차분함',
 '뻔뻔함',
 '어리숙함',
 '예민함',
 '예민함',
 '까칠함',
 '예민함',
 '까칠함',
 '우아함',
 '청초함',
 '우아함',
 '궁금함',
 '신선함',
 '평범함',
 '평범함',
 '평범함',
 '영리함',
 '화려함',
 '확고함',
 '따뜻함',
 '대단함',
 '신선함',
 '친숙함',
 '충분함',
 '화려함',
 '청아함',
 '탄탄함',
 '단단함',
 '사근사근함',
 '왕성함',
 '풍부함',
 '세심함',
 '끔찍함',
 '탄탄함',
 '다채함',
 '깔끔함',
 '진지함',
 '과감함',
 '발랄함',
 '유쾌함',
 '든든함',
 '청아함',
 '무난함',
 '드라마틱함',
 '섬세함',
 '불쌍함',
 '솔직함',
 '풋풋함',
 '화려함',
 '유쾌함',
 '솔직함',
 '담백함',
 '다정함',
 '뚜렷함',
 '기민함',
 '가혹함',
 '치열함',
 '소중함',
 '천진함',
 '달콤함',
 '만만함',
 '경쾌함',
 '우아함',
 '심심함',
 '발랄함',
 '쏠쏠함',
 '탄탄함',
 '심심함',
 '모호함',
 '유창함',
 '화려함',
 '어색함',
 '칼칼함',
 '익숙함',
 '상쾌함',
 '친근함',
 '경쾌함',
 '명확함',
 '따스함',
 '궁금함',
 '따뜻함',
 '흔함',
 '흔함',
 '중요함',
 '신선함',
 '강력함',
 '출중함',
 '평범함',
 '애틋함',
 '대견함',
 '친근함',
 '확고함',
 '강렬함',
 '강렬함',
 '발랄함',
 '청량함',
 '

- 최종 인물별 전체 키워드

In [ ]:
total_nj_list = nj_nva_meaningwords + nj_nxr_meaningwords
total_bp_list = bp_nva_meaningwords + bp_nxr_meaningwords

## 5) TF-IDF : Top20 인물 키워드 추출
- 형태소 유형을 형용사, 어근으로 제한하기 위해 문장 형태 아닌 키워드 형태로 TF-IDF 분석함

In [ ]:
!pip install konlpy

In [ ]:
from konlpy.tag import Kkma
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [ ]:
def extract_keywords_tfidf(text, num_keywords=5):
    kkma = Kkma()

    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform([text])
    feature_names = tfidf_vectorizer.get_feature_names_out()

    # TF-IDF 값에 따라 정렬된 단어들의 인덱스를 얻음
    sorted_indices = tfidf_matrix.toarray()[0].argsort()[::-1]

   # 상위 num_keywords 개의 단어와 해당 단어들의 TF-IDF 스코어 추출
    top_keywords = [(feature_names[idx], tfidf_matrix[0, idx]) for idx in sorted_indices[:num_keywords]]

    return top_keywords

In [ ]:
total_nj_string = ', '.join(total_nj_list)
total_bp_string = ', '.join(total_bp_list)


In [ ]:
total_nj_string

'폭넓음, 다름, 느림, 예쁨, 다름, 재미있음, 셈, 좋음, 좋음, 큼, 짧음, 예쁨, 다름, 예쁨, 재밌음, 뻔뻔함, 좋음, 큼, 기쁨, 큼, 재밌음, 강함, 다름, 좋음, 예쁨, 재밌음, 큼, 큼, 젊음, 재밌음, 재미있음, 느림, 큼, 큼, 큼, 좋음, 뻔뻔함, 재미있음, 다름, 좋음, 큼, 큼, 어림, 예쁨, 큼, 큼, 짧음, 짧음, 기쁨, 다름, 재미있음, 좋음, 큼, 재미있음, 힘들음, 좋음, 재밌음, 뻔함, 느림, 큼, 재밌음, 철없음, 큼, 재밌음, 좋음, 재밌음, 다름, 예쁨, 재밌음, 멋짐, 예쁨, 큼, 강렬함, 큼, 다름, 재밌음, 어림, 좋음, 길음, 짧음, 기쁨, 길음, 큼, 기쁨, 강렬함, 예쁨, 재미있음, 재미있음, 꾸밈없음, 좋음, 좋음, 좋음, 길음, 거칠음, 힘들음, 빠름, 큼, 끈끈함, 길음, 서툴음, 여림, 어림, 능글맞음, 좋음, 바쁨, 빠름, 빠름, 남다름, 힘들음, 좋음, 좋음, 격함, 큼, 묘함, 과감함, 큼, 색다름, 큼, 색다름, 좋음, 좋음, 밝음, 꾸준함, 깜찍함, 낯설음, 큼, 어림, 좋음, 꾸준함, 좋음, 빠름, 기쁨, 색다름, 빠름, 큼, 큼, 큼, 좋음, 좋음, 큼, 큼, 편함, 재미있음, 편함, 뚜렷함, 짧음, 좋음, 편함, 큼, 다름, 큼, 셈, 큼, 빠름, 색다름, 빠름, 짧음, 좋음, 빠름, 슬픔, 색다름, 화려함, 큼, 편함, 길음, 강함, 쿨함, 재미있음, 큼, 작음, 좋음, 빨갛음, 올바름, 다름, 올바름, 큼, 빨갛음, 검음, 큼, 밝음, 다름, 큼, 큼, 큼, 좋음, 좋음, 좋음, 다름, 흔함, 길음, 좋음, 좋음, 좋음, 유연함, 편함, 좋음, 좋음, 편함, 저렴함, 빠름, 젊음, 큼, 어림, 큼, 좋음, 강함, 아픔, 좋음, 좋음, 젊음, 힘들음, 다름, 큼, 큼, 따뜻함, 좋음, 바쁨, 바쁨, 친함, 좋음, 좋음, 평범함, 편함, 저렴함, 큼, 큼, 화려함, 큼, 큼, 아픔, 큼, 큼, 빠름, 아픔, 큼, 반함, 엄청남, 맑음, 훌륭함, 빠름, 좋음, 정갈함, 큼, 큼, 큼, 좋음

### 뉴진스 Top20

In [ ]:
text = nj_text

top_keywords_tfidf = extract_keywords_tfidf(text,num_keywords=20)

nj_tfidf_top20 = []

print("TF-IDF 기반 키워드 및 스코어 추출:")
for keyword, score in top_keywords_tfidf:
    nj_tfidf_top20.append(keyword)
    print(f"{keyword} - Score: {score}")

TF-IDF 기반 키워드 및 스코어 추출:
좋음 - Score: 0.7893283715186061
다름 - Score: 0.2416666159495292
빠름 - Score: 0.216746557230513
예쁨 - Score: 0.16632690354320115
색다름 - Score: 0.14662267106769997
멋짐 - Score: 0.12749797484147823
짧음 - Score: 0.12344122109652211
재밌음 - Score: 0.11764585860372764
어림 - Score: 0.11532771360660986
신선함 - Score: 0.11185049611093317
화려함 - Score: 0.10953235111381539
강렬함 - Score: 0.09794162612822646
다채함 - Score: 0.08866904613975532
재미있음 - Score: 0.08461229239479919
젊음 - Score: 0.0822941473976814
상큼함 - Score: 0.07939646615128418
남다름 - Score: 0.07765785740344583
길음 - Score: 0.0770783211541664
솔직함 - Score: 0.06838527741497469
밝음 - Score: 0.06548759616857745


### 블랙핑크 Top20

In [ ]:
text = total_bp_string

top_keywords_tfidf = extract_keywords_tfidf(text,num_keywords=20)

bp_tfidf_top20 = []

print("TF-IDF 기반 키워드 및 스코어 추출:")
for keyword, score in top_keywords_tfidf:
    bp_tfidf_top20.append(keyword)
    print(f"{keyword} - Score: {score}")

TF-IDF 기반 키워드 및 스코어 추출:
좋음 - Score: 0.8004518596445477
예쁨 - Score: 0.34134725485773076
우아함 - Score: 0.16843772837957893
다름 - Score: 0.1520411353514783
강렬함 - Score: 0.11924794929527711
화려함 - Score: 0.11477615119670423
남다름 - Score: 0.10285135626717651
멋짐 - Score: 0.0953983594362217
길음 - Score: 0.0909265613376488
밝음 - Score: 0.08943596197145784
빠름 - Score: 0.08794536260526688
강함 - Score: 0.08496416387288494
꾸준함 - Score: 0.07602056767573916
짧음 - Score: 0.07602056767573916
과감함 - Score: 0.0745299683095482
작음 - Score: 0.0745299683095482
어림 - Score: 0.07005817021097531
탄탄함 - Score: 0.07005817021097531
시원함 - Score: 0.06558637211240241
핫함 - Score: 0.06558637211240241


## 6) BPI 유사도 : 인물 키워드 분석
- BPI(Brand Personality Index) : 광고 모델 및 브랜드들의 개성을 나타내는 5개의 지표
- 5개의 지표: Sincerity, Excitement, Competence, Sophistication, Ruggedness
- 인물의 키워드와 5개의 지표 간의 유사도를 각각 구함
- 최종적으로 인물이 어떤 지표에 포함되는지 확인

### 뉴진스 키워드 분석

In [ ]:
nj_tfidf_top20

['좋음',
 '다름',
 '빠름',
 '예쁨',
 '색다름',
 '멋짐',
 '짧음',
 '재밌음',
 '어림',
 '신선함',
 '화려함',
 '강렬함',
 '다채함',
 '재미있음',
 '젊음',
 '상큼함',
 '남다름',
 '길음',
 '솔직함',
 '밝음']

In [ ]:
train_list = [keyword for keyword in nj_tfidf_top20]
sincerity = ["정직함","다정함","건전함","친근함","진실함","본래의","사실의"]
excitement = ["과감함","멋짐","상상력있음","독특함","유행선도적임","젊음","현대적임"]
competence = ["성공적임","믿음직함","지적임","안전함","전문적임","선두의"]
sophistication = ["매력적임","여성스러움","우아함","화려함","부드러움"]
ruggedness = ["거침","남성적임","외향적임","튼튼함"]

In [ ]:
train_list

['좋음',
 '다름',
 '빠름',
 '예쁨',
 '색다름',
 '멋짐',
 '짧음',
 '재밌음',
 '어림',
 '신선함',
 '화려함',
 '강렬함',
 '다채함',
 '재미있음',
 '젊음',
 '상큼함',
 '남다름',
 '길음',
 '솔직함',
 '밝음']

In [ ]:
from gensim.models import Word2Vec

docs = [train_list] + [sincerity] + [excitement] + [competence] + [sophistication] + [ruggedness]
model = Word2Vec(sentences=docs, vector_size=200, window=4, hs=1, negative=0, min_count=1, sg=1)

data = []
for keyword in train_list:
    columns = {'top20 키워드': keyword}
    for category, words in [('진실성', sincerity),
                            ('흥미로움', excitement),
                            ('유능함', competence),
                            ('세련됨', sophistication),
                            ('강인함', ruggedness)]:
        sim_sum = sum(model.wv.similarity(w1=keyword, w2=word) for word in words)
        columns[category] = sim_sum / len(words)
    data.append(columns)

df_data = pd.DataFrame(data)

df_data

,top20 키워드,진실성,흥미로움,유능함,세련됨,강인함
0,좋음,0.001880,-0.013159,-0.009533,0.070480,-0.011757
1,다름,-0.001187,-0.067770,-0.027513,-0.030692,-0.009812
2,빠름,0.009054,0.015446,-0.005051,-0.003453,0.007090
3,예쁨,-0.001493,0.019796,-0.061077,-0.050502,0.009698
4,색다름,0.037675,0.033805,-0.001723,-0.027754,-0.009042
5,멋짐,0.025672,0.189258,0.066629,-0.046929,-0.029039
6,짧음,0.012740,-0.061950,0.004611,0.012652,-0.015219
7,재밌음,0.012238,-0.039536,0.039635,0.003764,-0.017565
8,어림,0.040616,-0.000321,0.012792,-0.003898,-0.007042
9,신선함,0.066340,-0.026257,-0.051421,-0.010519,-0.012204


In [ ]:
df_data.mean(axis=0)

<ipython-input-237-6683c264f8f3>:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_data.mean(axis=0)


진실성     0.007552
흥미로움    0.016393
유능함     0.005803
세련됨     0.015869
강인함    -0.001610
dtype: float64

In [ ]:
df_data.mean(axis=0).idxmax()

<ipython-input-238-a81404cc11b5>:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_data.mean(axis=0).idxmax()


'흥미로움'

### 블랙핑크 키워드 분석

In [ ]:
bp_tfidf_top20

['좋음',
 '예쁨',
 '우아함',
 '다름',
 '강렬함',
 '화려함',
 '남다름',
 '멋짐',
 '길음',
 '밝음',
 '빠름',
 '강함',
 '꾸준함',
 '짧음',
 '과감함',
 '작음',
 '어림',
 '탄탄함',
 '시원함',
 '핫함']

In [ ]:
train_list = [keyword for keyword in bp_tfidf_top20]
sincerity = ["정직함","다정함","건전함","친근함","진실함","본래의","사실의"]
excitement = ["과감함","멋짐","상상력있음","독특함","유행선도적임","젊음","현대적임"]
competence = ["성공적임","믿음직함","지적임","안전함","전문적임","선두의"]
sophistication = ["매력적임","여성스러움","우아함","화려함","부드러움"]
ruggedness = ["거침","남성적임","외향적임","튼튼함"]

In [ ]:
from gensim.models import Word2Vec

docs = [train_list] + [sincerity] + [excitement] + [competence] + [sophistication] + [ruggedness]
model = Word2Vec(sentences=docs, vector_size=200, window=4, hs=1, negative=0, min_count=1, sg=1)

data = []
for keyword in train_list:
    columns = {'top20 키워드': keyword}
    for category, words in [('진실성', sincerity),
                            ('흥미로움', excitement),
                            ('유능함', competence),
                            ('세련됨', sophistication),
                            ('강인함', ruggedness)]:
        sim_sum = sum(model.wv.similarity(w1=keyword, w2=word) for word in words)
        columns[category] = sim_sum / len(words)
    data.append(columns)

df_data = pd.DataFrame(data)

df_data

,top20 키워드,진실성,흥미로움,유능함,세련됨,강인함
0,좋음,-0.002575,0.024669,0.055911,0.007531,-0.023493
1,예쁨,0.025041,0.029129,-0.018469,-0.008012,-0.013092
2,우아함,0.012975,0.019217,0.027434,0.211842,0.023178
3,다름,-0.010074,-0.000274,-0.028506,-0.051523,-0.009923
4,강렬함,0.000131,0.020964,-0.014827,0.009714,0.031461
5,화려함,0.026716,0.047323,0.034802,0.169900,0.003751
6,남다름,0.000728,-0.018976,-0.004349,0.000509,-0.016334
7,멋짐,-0.016062,0.175257,0.076799,0.032557,-0.016756
8,길음,-0.021987,-0.017750,0.037581,-0.025361,-0.000008
9,밝음,0.056315,0.004597,-0.004631,0.012234,0.020077


In [ ]:
df_data.mean(axis=0)

<ipython-input-242-6683c264f8f3>:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_data.mean(axis=0)


진실성     0.008060
흥미로움    0.024074
유능함     0.014668
세련됨     0.023931
강인함    -0.007586
dtype: float64

In [ ]:
df_data.mean(axis=0).idxmax()

<ipython-input-243-a81404cc11b5>:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_data.mean(axis=0).idxmax()


'흥미로움'

# 참고 -------------------------------

## 뉴스 크롤링 코드
- 검색어 : ex. 뉴진스
- 검색 시작일 : ex. 20230601
- 검색 종료일 : ex. 20230630
- 시작 페이지 : ex.1
- 종료 페이지 : ex.200


In [ ]:
#크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys

# 페이지 url 형식에 맞게 바꾸어 주는 함수 만들기
  #입력된 수를 1, 11, 21, 31 ...만들어 주는 함수
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:        return num+1
    else:
        return num+9*(num-1)

# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)

def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query="
        + search + "&pd=3&ds=" + start_date + "&de=" + end_date + "&start=" + str(start_page)
        print("생성url: ", url)
        return url
    else:
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query="
            + search + "&pd=3&ds=" + start_date + "&de=" + end_date + "&start=" + str(page)
            urls.append(url)
        print("생성url: ", urls)
        return urls

# html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)
def news_attrs_crawler(articles,attrs):
    attrs_content=[]
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content

# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

#html생성해서 기사크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    #html 불러오기
    original_html = requests.get(i,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver,'href')
    return url


#####뉴스크롤링 시작#####

#검색어 입력
search = input("검색할 키워드를 입력해주세요:")
#검색시작 날짜 입력
start_date = input("검색시작 날짜를 입력해주세요:")
#검색종료 날짜 입력
end_date = input("검색종료 날짜를 입력해주세요:")
#검색 시작할 페이지 입력
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 시작 페이지: ",page,"페이지")
#검색 종료할 페이지 입력
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 종료 페이지: ",page2,"페이지")


# naver url 생성
url = makeUrl(search,page,page2)

#뉴스 크롤러 실행
news_titles = []
news_url =[]
news_contents =[]
news_dates = []
for i in url:
    try:
        url = articles_crawler(url)
        news_url.append(url)
    except:
        print('기사를 건너뜁니다')


#제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist

#제목, 링크, 내용 담을 리스트 생성
news_url_1 = []

#1차원 리스트로 만들기(내용 제외)
makeList(news_url_1,news_url)

# NAVER 뉴스만 남기기
final_urls = []
for i in tqdm(range(len(news_url_1))):
    if "news.naver.com" in news_url_1[i]:
        final_urls.append(news_url_1[i])
    else:
        pass

# 뉴스 내용 크롤링

for i in tqdm(final_urls):
    #각 기사 html get하기
    news = requests.get(i,headers=headers)
    news_html = BeautifulSoup(news.text,"html.parser")

    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")

    # if search in title.get_text(): 타이들이 있는 경우


    # 뉴스 본문 가져오기
    content = news_html.select("div#dic_area")
    if content == []:
        content = news_html.select("#articeBody")

    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    news_titles.append(title)
    news_contents.append(content)

    try:
        html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time']
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1,repl='',string=str(news_date))
    # 날짜 가져오기
    news_dates.append(news_date)

print("검색된 기사 갯수: 총 ",(page2+1-page)*10,'개')
print("\n[뉴스 제목]")
print(news_titles)
print("\n[뉴스 링크]")
print(final_urls)
print("\n[뉴스 내용]")
print(news_contents)

print('news_title: ',len(news_titles))
print('news_url: ',len(final_urls))
print('news_contents: ',len(news_contents))
print('news_dates: ',len(news_dates))

###데이터 프레임으로 만들기###
import pandas as pd

#데이터 프레임 만들기
news_df = pd.DataFrame({'date':news_dates,'title':news_titles,'link':final_urls,'content':news_contents})

#중복 행 지우기
news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
print("중복 제거 후 행 개수: ",len(news_df))

#데이터 프레임 저장
now = datetime.datetime.now()
news_df.to_csv('{}_{}.csv'.format(search,now.strftime('%Y%m%d_%H시%M분%S초')),encoding='utf-8-sig',index=False)

## 블로그 크롤링 코드

## 용어리스트 2차 단어 보완 코드
- 2차 단어 보완 : KKM에서 추출한 형용사, 어근 형태소 리스트를 보완하기 위해,
- KKM에는 없고 크롤링한 데이터들에는 있었던 형태소들만 확인하여, 리스트에 추가함
- ex. 핫함, 쿨함 등의 신조어 및 성격어